In [ ]:
import numpy as np
from pydrake.planning import RobotDiagramBuilder, SceneGraphCollisionChecker
from cloth_tools.urdf.primitives import cylinder_urdf
from cloth_tools.drake.building import add_dual_ur5e_and_table_to_builder, add_meshcat_to_builder
from pydrake.math import RigidTransform, RollPitchYaw
from pydrake.visualization import ApplyVisualizationConfig, VisualizationConfig
from cloth_tools.ompl.dual_arm_planner import DualArmOmplPlanner

In [ ]:
robot_diagram_builder = RobotDiagramBuilder()
meshcat = add_meshcat_to_builder(robot_diagram_builder)
arm_left_index, arm_right_index = add_dual_ur5e_and_table_to_builder(robot_diagram_builder)

In [ ]:
plant = robot_diagram_builder.plant()

# These 4 lines are only for collision visualization
builder = robot_diagram_builder.builder()
plant.Finalize()
config = VisualizationConfig(publish_contacts=True, enable_alpha_sliders=True)
ApplyVisualizationConfig(config, builder=builder, plant=plant, meshcat=meshcat)

# A diagram is needed in the constructor of the SceneGraphCollisionChecker
# However, calling .Build() prevents us from adding more models, e.g. runtime obstacles
diagram = robot_diagram_builder.Build()

# Create default contexts ~= state
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(context)
diagram.ForcedPublish(context)

In [ ]:
collision_checker = SceneGraphCollisionChecker(
    model=diagram,
    robot_model_instances=[arm_left_index, arm_right_index],
    edge_step_size=0.125, # Arbitrary value: we don't use the CheckEdgeCollisionFree
    env_collision_padding = 0.005,
    self_collision_padding = 0.005,
)

In [ ]:
start_joints_left = np.deg2rad([30, -90, -90, -90, 90, 0])
start_joints_right = np.deg2rad([-136, -116, -110, -133, 40, 0])
# start_joints_left = np.deg2rad([90, -135, 95, -50, -90, -90])
# start_joints_right = np.deg2rad([-90, -45, -95, -130, 90, 90])

start_joints = np.concatenate([start_joints_left, start_joints_right])
collision_checker.CheckConfigCollisionFree(start_joints)

In [ ]:
plant.SetPositions(plant_context, arm_left_index, start_joints_left)
plant.SetPositions(plant_context, arm_right_index, start_joints_right)
diagram.ForcedPublish(context)

In [ ]:
home_joints_left = np.deg2rad([180, -135, 95, -50, -90, -90])
home_joints_right = np.deg2rad([-180, -45, -95, -130, 90, 90])

home_joints = np.concatenate([home_joints_left, home_joints_right])
collision_checker.CheckConfigCollisionFree(home_joints)

In [ ]:
plant.SetPositions(plant_context, arm_left_index, home_joints_left)
plant.SetPositions(plant_context, arm_right_index, home_joints_right)
diagram.ForcedPublish(context)

In [ ]:
planner = DualArmOmplPlanner(collision_checker.CheckConfigCollisionFree, max_planning_time=10)

In [ ]:
path = planner.plan_to_joint_configuration(start_joints_left, start_joints_right, home_joints_left, home_joints_right)

if path is None:
    print("No path found")
else:
    with np.printoptions(precision=3, suppress=True):
        print("Amount of waypoints:", len(path))
        print(path[0])
        print(path[-1])

In [ ]:
import time

total_time = 8.0

for dual_joint_configuration in path:
    left_joint_configuration, right_joint_configuration = dual_joint_configuration
    plant.SetPositions(plant_context, arm_left_index, left_joint_configuration)
    plant.SetPositions(plant_context, arm_right_index, right_joint_configuration)
    diagram.ForcedPublish(context)
    time.sleep(total_time / len(path))